In [1]:
import torch
from torch.nn import Conv2d, CrossEntropyLoss, Linear, MaxPool2d, Module
from torch.nn.functional import relu
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms.v2 import Compose, Grayscale, Resize, ToDtype, ToImage

In [2]:
TRAIN_PATH = "../dataset/train"
TEST_PATH = "../dataset/test"

In [3]:
transforms = Compose(
    [Grayscale(), Resize((176, 208)), ToImage(), ToDtype(torch.float32, scale=True)]
)

In [4]:
train_set = ImageFolder(root=TRAIN_PATH, transform=transforms)
test_set = ImageFolder(root=TEST_PATH, transform=transforms)

In [5]:
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = random_split(train_set, [train_size, val_size])

In [6]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [7]:
class CNN(Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = Linear(
            64 * 44 * 52, 128
        )  # Adjust based on your image size after pooling
        self.fc2 = Linear(
            128, 4
        )  # 4 classes: MildDemented, ModerateDemented, NonDemented, VeryMildDemented

    def forward(self, x):
        x = self.pool(relu(self.conv1(x)))
        x = self.pool(relu(self.conv2(x)))
        x = x.view(-1, 64 * 44 * 52)  # Flatten the tensor
        x = relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = CNN()
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [9]:
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0


for epoch in range(10):  # Number of epochs
    train(model, train_loader, criterion, optimizer, epoch)

[1, 100] loss: 1.181
[2, 100] loss: 0.894
[3, 100] loss: 0.850
[4, 100] loss: 0.725
[5, 100] loss: 0.603
[6, 100] loss: 0.489
[7, 100] loss: 0.375
[8, 100] loss: 0.281
[9, 100] loss: 0.162
[10, 100] loss: 0.100


In [10]:
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(
        f"Validation Loss: {val_loss / len(val_loader):.3f}, Accuracy: {100 * correct / total:.2f}%"
    )


validate(model, val_loader, criterion)

Validation Loss: 0.275, Accuracy: 89.76%


In [11]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


test(model, test_loader)

Test Accuracy: 59.50%


In [12]:
model_path = "../Models/alzheimers_cnn.pth"
torch.save(model.state_dict(), model_path)